# Set Up

In [1]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import matplotlib.image as mplimg
from matplotlib.pyplot import imshow

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf
import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from tensorflow.keras.layers import GlobalAveragePooling2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Model

import keras.backend as K
from keras.models import Sequential

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

import tensorflow as tf

In [27]:
import json

In [2]:
working_dir = "../../data/"

In [3]:
train_df = pd.read_csv(working_dir+"train.csv")
train_df.head(), len(train_df)

(           Image         Id
 0  0000e88ab.jpg  w_f48451c
 1  0001f9222.jpg  w_c3d896a
 2  00029d126.jpg  w_20df2c5
 3  00050a15a.jpg  new_whale
 4  0005c1ef8.jpg  new_whale,
 25361)

In [12]:
width, height = 96,96
target_size = (width,height)
epochs = 100
batch_size = 100
class_mode = 'categorical'

In [5]:
num_class = 28

In [6]:
img_path = "../../data/28_train"

In [7]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
#                                                             shear_range = 0.5,
                                                             horizontal_flip = True,
                                                             preprocessing_function = preprocess_input,
                                                             validation_split = 0.2,
                                                             rotation_range = 20,
                                                             width_shift_range = 0.2,
                                                             height_shift_range = 0.2)

In [8]:
train_set = train_datagen.flow_from_directory(directory = img_path,
                                                   target_size = target_size,
                                                   batch_size = batch_size,
                                                   class_mode = class_mode,
                                                   shuffle = True,
                                                   subset = 'training')
valid_set = train_datagen.flow_from_directory(directory = img_path,
                                                   target_size = target_size,
                                                   batch_size = batch_size,
                                                   class_mode = class_mode,
                                                   shuffle = True,
                                                   subset = 'validation')

Found 997 images belonging to 28 classes.
Found 238 images belonging to 28 classes.


# Grid Search Create Model

In [9]:
def model_generator():
    
    base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
        include_top=False, weights='imagenet', input_shape=(width, height, 3))
    
    inputs = Input(shape=(width, height, 3))
    x = base_model(inputs)
    
    for layer in base_model.layers:
        layer.trainable=False

    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dense(1024,activation="relu")(x)
    x = BatchNormalization()(x)
    x = Dense(1024,activation="relu")(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_class, activation='softmax')(x)
    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
        
    return model

In [10]:
early = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=10,
    verbose=1,
    restore_best_weights=True,
)

## Gridsearch manually

In [40]:
optimizers = ['Adam', 'Adamax', 'RMSprop', 'Nadam']
learn_rate = [0.01,0.001,0.0001]

#### optimizers

In [15]:
# optimizer tuning
histories = []
for opt in optimizers:
    print('\n')
    print(opt)
    print('-'*100)
    model = model_generator()
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    history = model.fit(train_set, 
                        epochs=100, 
                        batch_size=100, 
                        callbacks=[early],
                        validation_data=valid_set,
                        verbose=1)
    histories.append(history)



Adam
----------------------------------------------------------------------------------------------------
Epoch 1/100
10/10 [==============================] - 15s 1s/step - loss: 3.1584 - accuracy: 0.2237 - val_loss: 2.5328 - val_accuracy: 0.2857
Epoch 2/100
10/10 [==============================] - 12s 1s/step - loss: 1.6969 - accuracy: 0.5105 - val_loss: 2.0336 - val_accuracy: 0.4244
Epoch 3/100
10/10 [==============================] - 12s 1s/step - loss: 1.2830 - accuracy: 0.6018 - val_loss: 1.8040 - val_accuracy: 0.4790
Epoch 4/100
10/10 [==============================] - 12s 1s/step - loss: 0.9613 - accuracy: 0.7021 - val_loss: 1.7712 - val_accuracy: 0.4622
Epoch 5/100
10/10 [==============================] - 11s 1s/step - loss: 0.8949 - accuracy: 0.7252 - val_loss: 1.6633 - val_accuracy: 0.5126
Epoch 6/100
10/10 [==============================] - 12s 1s/step - loss: 0.6664 - accuracy: 0.7854 - val_loss: 1.4764 - val_accuracy: 0.5798
Epoch 7/100
10/10 [===========================

Epoch 35/100
10/10 [==============================] - 11s 1s/step - loss: 0.2269 - accuracy: 0.9248 - val_loss: 1.3145 - val_accuracy: 0.6429
Epoch 36/100
10/10 [==============================] - 11s 1s/step - loss: 0.2021 - accuracy: 0.9438 - val_loss: 1.4541 - val_accuracy: 0.6261
Epoch 37/100
10/10 [==============================] - 11s 1s/step - loss: 0.2091 - accuracy: 0.9358 - val_loss: 1.1976 - val_accuracy: 0.6975
Epoch 38/100
10/10 [==============================] - 12s 1s/step - loss: 0.2429 - accuracy: 0.9318 - val_loss: 1.4080 - val_accuracy: 0.6555
Epoch 39/100
10/10 [==============================] - 12s 1s/step - loss: 0.2177 - accuracy: 0.9338 - val_loss: 1.3782 - val_accuracy: 0.6471
Restoring model weights from the end of the best epoch.
Epoch 00039: early stopping


RMSprop
----------------------------------------------------------------------------------------------------
Epoch 1/100
10/10 [==============================] - 15s 1s/step - loss: 2.9562 - accuracy: 0.2

In [23]:
#histories_opt = [h.history for h in histories]
#with open('histories_optimizers.json', 'w') as fout:
#    json.dump(histories_opt, fout)

best optimizer: Adamax

#### learning rate

In [32]:
# learning rate tuning
histories = []
for lr in learn_rate:
    print('\n')
    print(f'learning_rate:{lr}')
    print('-'*100)
    model = model_generator()
    # change optimizer to the best
    opt = tf.keras.optimizers.Adamax(learning_rate=lr)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    history = model.fit(train_set, 
                        epochs=100, 
                        batch_size=100, 
                        callbacks=[early],
                        validation_data=valid_set,
                        verbose=1)
    histories.append(history)



learning_rate:0.01
----------------------------------------------------------------------------------------------------
Epoch 1/100
10/10 [==============================] - 15s 1s/step - loss: 5.9951 - accuracy: 0.1986 - val_loss: 9.3313 - val_accuracy: 0.3025
Epoch 2/100
10/10 [==============================] - 12s 1s/step - loss: 3.7477 - accuracy: 0.4453 - val_loss: 10.9799 - val_accuracy: 0.3025
Epoch 3/100
10/10 [==============================] - 11s 1s/step - loss: 3.1190 - accuracy: 0.5196 - val_loss: 8.1263 - val_accuracy: 0.3782
Epoch 4/100
10/10 [==============================] - 12s 1s/step - loss: 2.5102 - accuracy: 0.5827 - val_loss: 6.6416 - val_accuracy: 0.4118
Epoch 5/100
10/10 [==============================] - 12s 1s/step - loss: 1.8921 - accuracy: 0.6329 - val_loss: 4.2005 - val_accuracy: 0.5588
Epoch 6/100
10/10 [==============================] - 12s 1s/step - loss: 1.4089 - accuracy: 0.7041 - val_loss: 4.7967 - val_accuracy: 0.4538
Epoch 7/100
10/10 [============

10/10 [==============================] - 11s 1s/step - loss: 1.0765 - accuracy: 0.6710 - val_loss: 1.5113 - val_accuracy: 0.5672
Epoch 54/100
10/10 [==============================] - 12s 1s/step - loss: 1.0482 - accuracy: 0.6820 - val_loss: 1.4514 - val_accuracy: 0.5966
Epoch 55/100
10/10 [==============================] - 12s 1s/step - loss: 1.0699 - accuracy: 0.6720 - val_loss: 1.4326 - val_accuracy: 0.5798
Epoch 56/100
10/10 [==============================] - 12s 1s/step - loss: 1.0482 - accuracy: 0.6810 - val_loss: 1.4573 - val_accuracy: 0.5840
Epoch 57/100
10/10 [==============================] - 12s 1s/step - loss: 1.0023 - accuracy: 0.7041 - val_loss: 1.5030 - val_accuracy: 0.5462
Epoch 58/100
10/10 [==============================] - 11s 1s/step - loss: 0.9907 - accuracy: 0.7061 - val_loss: 1.4188 - val_accuracy: 0.5840
Epoch 59/100
10/10 [==============================] - 12s 1s/step - loss: 0.9549 - accuracy: 0.6951 - val_loss: 1.3809 - val_accuracy: 0.6513
Epoch 60/100
10/10 

In [33]:
#histories_lr = [h.history for h in histories]
#with open('grid_histories_lr.json', 'w') as fout:
#    json.dump(histories_lr, fout)

best lr: 0.0001

#### learning rate scheduler

In [37]:
# scheduler tuning
def exponential_decay_fn(epoch):
    return 0.01 * 0.1**(epoch / 20)

def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1**(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(lr0=0.0001, s=20)
scheduler_exp = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)

scheduler_perf = tf.keras.callbacks.ReduceLROnPlateau(factor=0.2, patience=5)

In [38]:
schedulers = [scheduler_exp,scheduler_perf]
histories = []
for sch in schedulers:
    model = model_generator()
    # change optimizer to the best
    opt = tf.keras.optimizers.Adamax(learning_rate=0.0001)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    history = model.fit(train_set, 
                        epochs=100, 
                        batch_size=100, 
                        callbacks=[early,sch],
                        validation_data=valid_set,
                        verbose=1)
    histories.append(history)

Epoch 1/100
10/10 [==============================] - 15s 1s/step - loss: 4.0420 - accuracy: 0.0391 - val_loss: 3.4521 - val_accuracy: 0.0462
Epoch 2/100
10/10 [==============================] - 12s 1s/step - loss: 3.6469 - accuracy: 0.0692 - val_loss: 3.2984 - val_accuracy: 0.0756
Epoch 3/100
10/10 [==============================] - 11s 1s/step - loss: 3.5452 - accuracy: 0.1013 - val_loss: 3.2003 - val_accuracy: 0.1218
Epoch 4/100
10/10 [==============================] - 12s 1s/step - loss: 3.3075 - accuracy: 0.1284 - val_loss: 3.1018 - val_accuracy: 0.1681
Epoch 5/100
10/10 [==============================] - 12s 1s/step - loss: 3.1915 - accuracy: 0.1444 - val_loss: 2.9517 - val_accuracy: 0.2227
Epoch 6/100
10/10 [==============================] - 12s 1s/step - loss: 3.0729 - accuracy: 0.1846 - val_loss: 2.9199 - val_accuracy: 0.2353
Epoch 7/100
10/10 [==============================] - 12s 1s/step - loss: 3.0348 - accuracy: 0.1825 - val_loss: 2.9031 - val_accuracy: 0.1975
Epoch 8/100
1

Epoch 59/100
10/10 [==============================] - 11s 1s/step - loss: 2.5286 - accuracy: 0.2959 - val_loss: 2.4949 - val_accuracy: 0.3235
Epoch 60/100
10/10 [==============================] - 12s 1s/step - loss: 2.4909 - accuracy: 0.2969 - val_loss: 2.4068 - val_accuracy: 0.3235
Epoch 61/100
10/10 [==============================] - 12s 1s/step - loss: 2.4680 - accuracy: 0.2869 - val_loss: 2.4136 - val_accuracy: 0.3445
Epoch 62/100
10/10 [==============================] - 12s 1s/step - loss: 2.5367 - accuracy: 0.2959 - val_loss: 2.4129 - val_accuracy: 0.3445
Epoch 63/100
10/10 [==============================] - 12s 1s/step - loss: 2.6343 - accuracy: 0.2728 - val_loss: 2.3411 - val_accuracy: 0.3403
Epoch 64/100
10/10 [==============================] - 12s 1s/step - loss: 2.5070 - accuracy: 0.3029 - val_loss: 2.4344 - val_accuracy: 0.3277
Epoch 65/100
10/10 [==============================] - 11s 1s/step - loss: 2.5047 - accuracy: 0.3029 - val_loss: 2.4361 - val_accuracy: 0.3992
Epoch 

In [39]:
#histories_sch = [h.history for h in histories]
#with open('./history/grid_histories_scheduler.json', 'w') as fout:
#    json.dump(histories_lr, fout)

performance scheduler is better than exponential scheduler, but is worse than no scheduler.

In [ ]:
# xxx = best scheduler
history = histories[xxx]

plt.plot(history.epoch, history.history["lr"], "o-")
plt.axis([0, n_epochs - 1, 0, 0.011])
plt.xlabel("Epoch")
plt.ylabel("Learning Rate")
plt.title("Exponential Scheduling", fontsize=14)
plt.grid(True)
plt.show()

-------------------------

# Optimizing Hyper-parameters by  Grid Search


In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier

## Tune Optimization 


In [ ]:
# with the best epoch and batch_size
best_epochs=100
#best_batch_size = 100

model = KerasClassifier(build_fn=create_model, epochs=best_epochs, verbose=1, callbacks=[early])
# define the grid search parameters
optimizer = [ 'Adam', 'Adamax','Adamaw', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

Epoch 1/10
7/7 [==============================] - 7s 696ms/step - loss: 7.9830 - accuracy: 0.2235
Epoch 2/10
7/7 [==============================] - 5s 692ms/step - loss: 6.6134 - accuracy: 0.4010
Epoch 3/10
7/7 [==============================] - 5s 710ms/step - loss: 5.9269 - accuracy: 0.4090
Epoch 4/10
7/7 [==============================] - 5s 689ms/step - loss: 5.1590 - accuracy: 0.4137
Epoch 5/10
7/7 [==============================] - 5s 679ms/step - loss: 5.1102 - accuracy: 0.3925
Epoch 6/10
7/7 [==============================] - 5s 678ms/step - loss: 4.7999 - accuracy: 0.3701
Epoch 7/10
7/7 [==============================] - 5s 675ms/step - loss: 4.2015 - accuracy: 0.4367
Epoch 8/10
7/7 [==============================] - 5s 665ms/step - loss: 3.9610 - accuracy: 0.4330
Epoch 9/10
7/7 [==============================] - 5s 668ms/step - loss: 4.0467 - accuracy: 0.4120
Epoch 10/10
7/7 [==============================] - 5s 675ms/step - loss: 3.8148 - accuracy: 0.4236


In [ ]:
grid_result = grid.fit(train_generator)

In [ ]:
grid_result.best_score_, grid_result.best_params_

NameError: ignored

## Tune Learning Rate


In [ ]:
model2 = KerasClassifier(build_fn=create_model, epochs=best_epochs, verbose=1, callbacks=[early])
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.05]
#momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
param_grid = dict(learn_rate=learn_rate)
grid2 = GridSearchCV(estimator=model2, param_grid=param_grid, n_jobs=-1, cv=3)
grid2_result = grid2.fit(train_generator)

In [ ]:
grid2_result.best_score_, grid2_result.best_params_